#1 Чем плохи деструкторы в питоне?

Касательно деструкторов:

  1. Деструктор будет работать не корректно в случае круговой ссылки. 
Что это значит. Это значит, что один класс ссылается на другой, котороый ссылается на первый
(пример ниже)

In [11]:
class Vehicle():
  def __init__(self, id:int, car):
    self.id = id
    self.manufacturer = car
    print(f'Vehicle with id: {self.id} by {self.manufacturer} created!')

  def __del__(self):
    print(f'Vehicle with id: {self.id} by {self.manufacturer} destroyed!')

class Car():
  def __init__(self, id):
    self.id = id
    self.manufacturer = Vehicle(id, self)
    print(f'Car with id: {self.id} by {self.manufacturer} created!')

  def __del__(self):
    print(f'Car with id: {self.id} by {self.manufacturer} destroyed!')

In [12]:
polo = Car(1)
del polo
#Как можно видеть, экземпляры создались, но не были удалены
#И при этом деструктор не был вызван

Vehicle with id: 1 by <__main__.Car object at 0x7f0304698100> created!
Car with id: 1 by <__main__.Vehicle object at 0x7f0304698580> created!


  2. Деструктор так же не будет работать в том случае, если возникает вызов исключения про сборке экземпляра (в init). То есть, если в конструкторе вызывается обработчик ошибок, то будет вызван метод деструктора. Но на самом деле объект создан не будет (пример ниже)

In [13]:
class Car():
  def __init__(self, speed):
    if speed > 320:
      raise Exception('Too mach too young too fast!')
    else:
      print('New car created')

  def __del__(self):
    print('Cleaning resources')

In [14]:
octavia = Car(230)# Тут все прошло нормально, так как 230 < 320
gt_911 = Car(350)

#Ну и да. Оно упало.

New car created


Exception: ignored

#2. Почему может быть плох import в начале файла?

Вспоминая, что в python существует такая штука как PEP8, согласно которой импорты нужно располагать в начале файла. Чем может быть плох import в начале файла...ну на ум приходит пока что только условные импорты.
К примеру, если у нас есть причниы, по которой одна из версий модуля не работает в определенной операционной системе, или версии самого python. Так вот, можно выполнять импорты в зависимости от выполнения условий (пример ниже)

In [15]:
#Пример простой, но для демонстрации сойдет.

purpose = str(input("Выберите visualisation или calculating: "))

purpose_list = ['visualisation', 'calculating']

if purpose in purpose_list:
  if purpose == purpose_list[0]:
    import pandas as module
    print('Импортирован модуль визуализации')
  if purpose == purpose_list[1]:
    import numpy as module
    print('Импортирован модуль для вычислений')


Выберите visualisation или calculating: calculating
Импортирован модуль для вычислений


In [16]:
%pip freeze -> requriments.txt

In [17]:
with open('requriments.txt', 'r', encoding='UTF-8')as f:
  f = f.read()
  if purpose == purpose_list[0]:
    print('pandas' in f)
  if purpose == purpose_list[1]:
    print('numpy' in f)
#Вывод будет true в любом случае, если использовать google colab, так как в нем все модули уже установлены...

True


#3. Как в питоне «вывалиться» из любой вложенности или рекурсии по аналогу «goto:»?

В питоне нет привычного или прямого аналога goto:, но есть условные операторы, такие как if, elif, else. Также есть методы break и continue. If else - понятно, если что-то то делаем что-то, если нет, то иначе. elif - это альтернатива if. То есть, если if условие не выполнилось, будет проверяться условие elif'а. 


С break и continue все тоже блоее менее ясно из названия. Первый прервет выполнение цикла или функции, вотрой продолжит, если условие выполнилось.

Также можно использовать механизм флагов. Объямить преременную, в которой задать определенное значение (чаще всего True или False), а затем добавить к условию проверку флага. Как только необходимое условие будет выполнено, то заменяем значение флага на противоположное или на то, которе не пройдет условие проверки.

In [18]:
def factorial_recursive(n):
    if n == 1:
        return n
    elif n > 10:
        return n*factorial_recursive(n-1)
    else: return n + 1

factorial_recursive(8)

9

#4.  Сервер отдает сообщение один раз (1кб, бинарь). Его необходимо сохранить в базе, нельзя потерять(!). Как бы вы решали такую задачу?

Для данной задачи вариантов решения много, так как не достаточно данных. 

Почему сообщение можно потерять в данной ситуации?
- Сообщение было пропущено обработчиком
- При синхронной передаче сообщения в бд, база перестала отвечать из-за большой нагрузки
- При обращении к базе данных, сервер на котором она располагается, был отключен
- Файл с базой данных на локальном хосте был перемещен в другое место
- Программа для отлова сообщение может выдать ошибку до того как сообщение будет записано
- Запрос в базу на добавление сообщения может быть долгим и за это время может прийти другое сообщение.

Как бы это решить? Сейчас подумаем.
- Стоит разграничить функции. Одна функция будет принимать сообщения, вторая производить запись, третья будет проверять наличие сообщения и на всякий случай делать запись в плозкий файл и очищать его от уже записанных сообщений, а четвертая будет проверять доступность хранилища для записи.

- Также использовать синхронную передачу в базу не лучшая идея, так как при большой частоте сообщений и медленной базе, мы точно будем терять сообщения. Лучше использовать асинхронную передачу, либо ORM транзакции. А если есть возможность, то вообще сессии из модуля SQLAlchemy. Они работают как транзакции, но открываются перед записью в бд, а также гарантируют, что записи будут сохранены в бд в единственном экземпляре

- Можно хранить сообщения в небольшом локальном хранилище, откуда их можно будет передавать в основную базу.

- Также стоит использовать многопоточность, чтобы не создавать проблемы при большом наплыве сообщений, если таковы будут.


*Если будет нужно, выше описанное раскидать по потокам, чтобы увеличить скорость и при множественных запросах это не создаст очередь.


#5. Есть хранилище данных, которое обновляется раз в сутки, SQL или ElasticSearch (можно не знать про ES). Пользователь работает с хранилищем. Запрос выполнялся 30+ секунд. Результат корректный. Но этот же запрос может быть повторен другим пользователем. Как ускорить получение ответов от хранилища?

Для оптимизации запросов в базу данных часто применяют разные методы оптимизации
- Механизм кэширования. Этот механизм можно использовать как для кэширования результатов запросов, так и для создания зеркал базы данных (но тогда нужно будет учитывать множество нюансов, которые возникнут при данном подходе). Но при использовании кэширования нужно учитывать, что должна существовать политика обновления этого самого кэша. 

- Индексация запросов. Этот механизм позволяет базе данных не проходить по каждой таблице и каждой строке в поиске нужного результата, а переходить сразу к нужному месту, откуда и начинается выполнение запроса. Это позволит сократить время множества запросов, даже вложенных.

- Использование ORM сессий. Сессия в ORM SQLAlchemy это инструмент, который работает схожим образом с транзакциями, но открывает их непосредственно перед записью в бд. Также это позволяет ускорить запросы в некоторых случаях. В том числе сессии позволяют поддерживать корректные и актуальные данные, так как есть использование механизма кэширования. Разве что с политикой обновления нужно разобраться, так как она прописана изначально одна, а нам может потребоваться её изменить.

Подводя итоги, хочу отметить, что для каждой ситуации существует свое решение. Универсального решения всех проблем не существует, но три примера приведенные выше используются достаточно часто, а значит, доверие к этим методам есть.

# Вопрос по SQL

В чем может быть отличие использование LIKEот MATCH… AGAINST?

Например в запросах

 

SELECT * FROM cache WHERE content like ('%future%' and '%trends%')

 

И

 

SELECT *  FROM cache WHERE  MATCH (content) AGAINST ( '+future +trends' IN BOOLEAN MODE)

Оператор LIKE используется для посика данных столбца, которые соответствуют шаблону. То есть этот оператор работает как регулярные выражения, которые ищут подходящие по шаблону данные

В приведенном запросе идет поиск вех значений которые имеют "future" и "trends"

Что по поводу MATCH-AGAINST. С ним ранее не сталкивался, но как я понял из информации в сети - он ищет конкретные слова и производит их сортировку по релевантности. Что касательно режима BOOLEAN - в этом режиме можно указывать логические операторы. Так же в этом режиме нет автоматической сортировки